In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

H = SparsePauliOp.from_list([('II', -1), ('IZ', 0.3), ('XI', -0.3), ('ZY', -0.01), ('YX', 0.1)])
Hmat = H.to_matrix()

eig = np.linalg.eigvals(Hmat)
print('NUMPY eig',eig)

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=H)
print('VQE eig', result.eigenvalue)

NUMPY eig [-0.39328755+2.04391586e-17j -0.89      +1.90735584e-16j
 -1.11      +7.23350057e-17j -1.60671245-1.10037401e-16j]
VQE eig -1.6067124153122894


In [9]:

n_qpe_qbits = 10

U = HamiltonianGate(Hmat, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
#measure_circ.draw("mpl")


In [31]:
# from qiskit.primitives import StatevectorSampler,PrimitiveJob
# from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# from qiskit.visualization import plot_histogram
# from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
# from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
# from qiskit_aer import AerSimulator
# sampler = StatevectorSampler()
# pub = (measure_circ)
# job_sampler = sampler.run([pub], shots=100)

# result_sampler = job_sampler.result()
#counts_sampler = result_sampler[0].data.meas.get_counts()

#print(counts_sampler)

In [11]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

{'0110000010': 4, '1010110100': 2, '1000100100': 3, '1110000010': 1}


In [12]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

MAX count: ('0110000010', 4)
Theta value: 0.255859375
QPE-approximated U-eigenvalue: (-0.036807222941358866+0.9993223845883495j)
QPE-approximated H-eigenvalue: -1.607611865704152
